# Pollard's Kangaroo ECDLP Solver for SECP256K1

## Overview

This notebook demonstrates the **Kangaroo** algorithm - a high-performance solver for the Elliptic Curve Discrete Logarithm Problem (ECDLP) on the SECP256K1 curve. The implementation supports:

- **Multi-threaded CPU computation**
- **GPU acceleration** (CUDA-enabled NVIDIA GPUs)
- **Distributed computing** (client/server architecture)
- **Work file management** for checkpoint/resume capabilities

### Key Features

- Fixed size arithmetic
- Fast Modular Inversion (Delayed Right Shift 62 bits)
- SecpK1 Fast modular multiplication
- Multi-GPU support
- CUDA optimization via inline PTX assembly

### Limitations

This program is limited to a **125-bit interval search**.

---

## What is the ECDLP Problem?

The Elliptic Curve Discrete Logarithm Problem is: Given points P and Q on an elliptic curve, find the scalar k such that:

```
Q = k × P
```

For Bitcoin (SECP256K1), this problem is computationally hard, forming the basis of cryptocurrency security. However, if the search space is limited to a known range `[k1, k2]`, Pollard's Kangaroo algorithm can find k efficiently.

### Algorithm Overview

The Kangaroo algorithm uses two "herds" of kangaroos:
- **Tame kangaroos**: Start from known positions within the range
- **Wild kangaroos**: Start from the target public key

When a tame and wild kangaroo collide (meet at the same point), the private key can be computed. The algorithm requires approximately `2.08 × √(k2-k1)` group operations on average.

## Section 1: Environment Setup

### Prerequisites

For CPU-only builds:
- GCC compiler (version 7 or higher)
- Make build tool

For GPU builds (optional but recommended for performance):
- NVIDIA GPU with CUDA support
- CUDA SDK (version 8.0 or higher)
- NVIDIA drivers

### Clone the Repository

In [ ]:
# Clone the Kangaroo repository
!git clone https://github.com/drqsatoshi/kangaroo.git
%cd kangaroo
!ls -la

## Section 2: Compilation

### CPU-Only Build (Works on Kaggle)

This build is suitable for Kaggle environments and systems without GPU support:

In [ ]:
# Clean previous builds
!make clean

# Build CPU version
!make all -j4

### GPU Build (For CUDA-enabled systems)

**Note**: Skip this section if you're running on Kaggle or don't have NVIDIA GPU.

For systems with NVIDIA GPUs and CUDA toolkit:

In [ ]:
# Check CUDA availability (uncomment to run)
# !nvcc --version
# !nvidia-smi

# Build with GPU support
# Adjust ccap based on your GPU:
# - Compute 7.5 (T4): ccap=75
# - Compute 8.0 (A100): ccap=80
# - Compute 8.6 (RTX 3090): ccap=86
# - Compute 8.9 (L4): ccap=89

# !make clean
# !make gpu=1 CUDA=/usr/local/cuda CXXCUDA=/usr/bin/g++ ccap=75 all -j4

## Section 3: Understanding Input Files

Kangaroo requires an input configuration file with the following structure:

```
Start_Range (hex)
End_Range (hex)
Public_Key_1 (hex, compressed or uncompressed)
Public_Key_2 (optional)
...
```

Let's examine some example input files:

In [ ]:
# View the simple example input file
!cat in.txt

In [ ]:
# View puzzle #120 configuration
!cat puzzle120.txt

In [ ]:
# View the first few puzzles from the puzzle file
!head -30 puzzle32.txt

## Section 4: Basic Usage Examples

### Example 1: Solving a Small Range (in.txt)

This example demonstrates solving a small 56-bit range, which should complete quickly on CPU:

In [ ]:
# Run Kangaroo on the simple example with 4 CPU threads
# This should complete in under a minute
!./kangaroo -t 4 in.txt

### Example 2: Using the Puzzle Extractor

The `-puzzle` option automatically extracts a specific puzzle from the puzzle file:

In [ ]:
# This creates puzzle120.txt automatically and runs it
# WARNING: Puzzle #120 is VERY large (119-bit range) and won't complete on CPU
# We'll run it for just a few seconds to see the output

import subprocess
import signal

# Run for 10 seconds then terminate
try:
    process = subprocess.Popen(['./kangaroo', '-t', '4', '-d', '20', '-puzzle', '120', 'puzzle32.txt'],
                              stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(timeout=10)
    print(stdout)
    if stderr:
        print("STDERR:", stderr)
except subprocess.TimeoutExpired:
    process.kill()
    stdout, stderr = process.communicate()
    print("Process terminated after 10 seconds")
    print(stdout)
    print("\nNote: Puzzle #120 requires GPU and weeks of computation time to solve.")

## Section 5: Command Line Options

### Essential Options

| Option | Description |
|--------|-------------|
| `-t N` | Number of CPU threads to use |
| `-d N` | Distinguished point bits (controls memory/performance tradeoff) |
| `-gpu` | Enable GPU acceleration |
| `-gpuId 0,1,...` | Specify which GPU(s) to use |
| `-puzzle N file` | Extract and run puzzle #N from puzzle file |

### Work File Management

| Option | Description |
|--------|-------------|
| `-w file` | Save work to file |
| `-i file` | Load work from file (resume) |
| `-wi N` | Auto-save interval in seconds |
| `-ws` | Save kangaroos in work file |
| `-winfo file` | Display work file information |

### Advanced Options

| Option | Description |
|--------|-------------|
| `-s` | Start in server mode |
| `-c IP` | Connect to server at IP |
| `-sp PORT` | Server port (default: 17403) |
| `-o file` | Output results to file |

## Section 6: Creating Custom Input Files

Let's create a custom input file for a small puzzle we can actually solve:

In [ ]:
# Create a custom 40-bit puzzle (manageable on CPU)
with open('custom_puzzle.txt', 'w') as f:
    f.write("8000000000\n")  # Start: 2^39
    f.write("FFFFFFFFFF\n")  # End: 2^40-1
    # Example compressed public key
    f.write("02E9F43F810784FF1E91D8BC7C4FF06BFEE935DA71D7350734C3472FE305FEF82A\n")

!cat custom_puzzle.txt

## Section 7: Understanding Distinguished Points (DP)

The Distinguished Point method is crucial for performance:

- Instead of storing all kangaroo positions, only store points with x-coordinate starting with `dpBit` zero bits
- This dramatically reduces memory usage
- Trade-off: More DP bits = Less memory, but more overhead

### DP Overhead Table

| nbKangaroo × 2^dpBit / √N | DP Overhead | Average Operations |
|---------------------------|-------------|-------------------|
| 4.000 | ~71.0% | 3.55 √N |
| 2.000 | ~44.2% | 2.99 √N |
| 1.000 | ~26.0% | 2.62 √N |
| 0.500 | ~14.5% | 2.38 √N |
| 0.250 | ~7.7% | 2.24 √N |
| 0.125 | ~4.0% | 2.16 √N |

### Calculating Expected Operations

In [ ]:
import math

def calculate_expected_ops(range_bits):
    """
    Calculate expected operations for Pollard's Kangaroo algorithm.
    
    Args:
        range_bits: Number of bits in the search range
    
    Returns:
        Tuple of (expected_ops, sqrt_n_bits)
    """
    sqrt_n_bits = range_bits / 2
    # Kangaroo needs approximately 2.08 * sqrt(N) operations
    expected_ops_bits = math.log2(2.08) + sqrt_n_bits
    return 2**expected_ops_bits, sqrt_n_bits

# Examples
for bits in [40, 56, 64, 84, 110, 115, 120]:
    ops, sqrt_bits = calculate_expected_ops(bits)
    print(f"Range: {bits} bits | Expected ops: 2^{sqrt_bits + math.log2(2.08):.2f} | {ops:.2e}")

## Section 8: Work File Operations

Work files allow you to:
1. Save progress periodically
2. Resume interrupted searches
3. Merge work from multiple machines

### Creating a Work File with Auto-Save

In [ ]:
# Example: Run with work file saving every 30 seconds
# (Commented out to avoid long-running processes)

# !./kangaroo -t 4 -ws -w progress.work -wi 30 in.txt

### Viewing Work File Information

In [ ]:
# If you have a work file, view its info:
# !./kangaroo -winfo progress.work

print("Work file info shows:")
print("- DP bits used")
print("- Search range (start/stop)")
print("- Public key being searched")
print("- Operations count")
print("- Time elapsed")
print("- Hash table statistics")
print("- Number of kangaroos")

### Merging Work Files

In [ ]:
# Merge two work files (will solve key if collision found)
# !./kangaroo -wm work1.work work2.work merged.work

## Section 9: Distributed Computing Setup

Kangaroo supports client-server architecture for distributed solving:

### Server Setup

```bash
# Start server with 12 DP bits, saving every 5 minutes
./kangaroo -s -d 12 -w save.work -wi 300 -o result.txt in.txt
```

### Client Setup

```bash
# Connect client to server (replace 'server_ip' with actual IP)
./kangaroo -t 4 -gpu -w client.work -wi 600 -c server_ip
```

### Architecture Diagram

```
┌──────────────┐
│    Server    │  <- Manages hash table, detects collisions
│  (Kangaroo)  │     Saves work files periodically
└──────┬───────┘
       │
   ┌───┴────┬──────────┬──────────┐
   │        │          │          │
┌──▼───┐ ┌─▼────┐ ┌──▼───┐  ┌──▼───┐
│Client│ │Client│ │Client│  │Client│  <- Generate kangaroos
│ CPU  │ │ GPU  │ │ GPU  │  │ GPU  │     Send DPs to server
└──────┘ └──────┘ └──────┘  └──────┘
```

## Section 10: Performance Analysis

### Historical Performance Data

The Kangaroo solver has successfully solved several Bitcoin puzzles:

| Puzzle | Bits | Prize | Time | Hardware | Operations |
|--------|------|-------|------|----------|------------|
| #110 | 109 | 1.10 BTC | 2.1 days | 256× Tesla V100 | 2^55.55 |
| #115 | 114 | 1.15 BTC | 13 days | 256× Tesla V100 | 2^58.36 |

### Performance Calculator

In [ ]:
def estimate_time(range_bits, mkeys_per_sec):
    """
    Estimate time to solve based on performance.
    
    Args:
        range_bits: Search range in bits
        mkeys_per_sec: Million keys per second (MKey/s)
    
    Returns:
        Estimated time in seconds
    """
    expected_ops, _ = calculate_expected_ops(range_bits)
    ops_per_sec = mkeys_per_sec * 1e6
    time_seconds = expected_ops / ops_per_sec
    return time_seconds

def format_time(seconds):
    """Convert seconds to human-readable format."""
    if seconds < 60:
        return f"{seconds:.1f} seconds"
    elif seconds < 3600:
        return f"{seconds/60:.1f} minutes"
    elif seconds < 86400:
        return f"{seconds/3600:.1f} hours"
    elif seconds < 31536000:
        return f"{seconds/86400:.1f} days"
    else:
        return f"{seconds/31536000:.1f} years"

# Example performance estimates
print("Performance Estimates:\n")
print("CPU (4 cores, ~20 MKey/s):")
for bits in [40, 50, 56, 60]:
    time_sec = estimate_time(bits, 20)
    print(f"  {bits}-bit range: {format_time(time_sec)}")

print("\nSingle Tesla V100 (~2000 MKey/s):")
for bits in [60, 70, 80, 84]:
    time_sec = estimate_time(bits, 2000)
    print(f"  {bits}-bit range: {format_time(time_sec)}")

print("\n256x Tesla V100 (~500,000 MKey/s):")
for bits in [100, 110, 115, 120]:
    time_sec = estimate_time(bits, 500000)
    print(f"  {bits}-bit range: {format_time(time_sec)}")

## Section 11: Visualization

### Probability of Success Over Time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Probability of success function
def success_probability(operations, sqrt_n):
    """
    Calculate probability of finding collision.
    Based on birthday paradox for Pollard's Kangaroo.
    """
    # Simplified model: P ≈ 1 - exp(-ops^2 / (2*N))
    # where ops is in terms of sqrt(N)
    ratio = operations / sqrt_n
    return 1 - np.exp(-ratio**2 / 2)

# Create plot
x = np.linspace(0, 4, 100)
y = success_probability(x, 1)

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'b-', linewidth=2)
plt.axvline(x=2.08, color='r', linestyle='--', label='Expected (2.08√N)')
plt.axhline(y=0.5, color='g', linestyle='--', alpha=0.5, label='50% probability')
plt.grid(True, alpha=0.3)
plt.xlabel('Operations (in terms of √N)', fontsize=12)
plt.ylabel('Probability of Success', fontsize=12)
plt.title('Pollard\'s Kangaroo: Probability of Success vs. Operations', fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig('success_probability.png', dpi=150, bbox_inches='tight')
plt.show()

print("Key points:")
print(f"  50% chance at: {x[np.argmin(np.abs(y - 0.5))]:.2f}√N operations")
print(f"  90% chance at: {x[np.argmin(np.abs(y - 0.9))]:.2f}√N operations")
print(f"  99% chance at: {x[np.argmin(np.abs(y - 0.99))]:.2f}√N operations")

### Range Size vs. Computation Time

In [ ]:
# Visualize how computation time scales with range size
range_bits = np.arange(40, 130, 5)
times_cpu = [estimate_time(b, 20) for b in range_bits]
times_gpu = [estimate_time(b, 2000) for b in range_bits]
times_cluster = [estimate_time(b, 500000) for b in range_bits]

plt.figure(figsize=(12, 7))
plt.semilogy(range_bits, np.array(times_cpu)/3600, 'o-', label='CPU (4 cores, 20 MKey/s)', linewidth=2)
plt.semilogy(range_bits, np.array(times_gpu)/3600, 's-', label='Tesla V100 (2000 MKey/s)', linewidth=2)
plt.semilogy(range_bits, np.array(times_cluster)/3600, '^-', label='256× Tesla V100 (500k MKey/s)', linewidth=2)

plt.axhline(y=24, color='gray', linestyle='--', alpha=0.5, label='1 day')
plt.axhline(y=24*30, color='gray', linestyle=':', alpha=0.5, label='1 month')
plt.axhline(y=24*365, color='gray', linestyle='-.', alpha=0.5, label='1 year')

plt.grid(True, alpha=0.3)
plt.xlabel('Search Range (bits)', fontsize=12)
plt.ylabel('Estimated Time (hours, log scale)', fontsize=12)
plt.title('Kangaroo Algorithm: Range Size vs. Computation Time', fontsize=14)
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig('range_vs_time.png', dpi=150, bbox_inches='tight')
plt.show()

## Section 12: Troubleshooting

### Common Issues and Solutions

#### 1. CUDA/GPU Issues

**Problem**: `nvcc: command not found`
- **Solution**: Install CUDA SDK or build CPU-only version with `make all`

**Problem**: GPU not detected
- **Solution**: Check with `nvidia-smi` and ensure drivers are installed

#### 2. Compilation Errors

**Problem**: `undefined reference to pthread_create`
- **Solution**: Ensure you have proper GCC version (>= 7.0)

**Problem**: Compute capability mismatch
- **Solution**: Adjust `ccap` parameter in Makefile to match your GPU

#### 3. Runtime Issues

**Problem**: "Out of memory" errors
- **Solution**: Increase DP bits with `-d` option (e.g., `-d 20`)

**Problem**: Very slow progress
- **Solution**: Range too large for available hardware - reduce range or add more GPUs

#### 4. Input File Errors

**Problem**: "Invalid public key"
- **Solution**: Ensure public key is in hex format (compressed: starts with 02/03, 66 chars; uncompressed: starts with 04, 130 chars)

**Problem**: "Range width too large"
- **Solution**: This program is limited to 125-bit ranges

## Section 13: Security and Ethical Considerations

### Important Notes

⚠️ **Warning**: This tool is for:
- **Educational purposes** - Understanding elliptic curve cryptography
- **Research** - Analyzing cryptographic strength
- **Authorized testing** - Only on systems/keys you own or have permission to test

### Legal Disclaimer

- DO NOT use this tool to attack real Bitcoin addresses or wallets you don't own
- Unauthorized access to cryptocurrency wallets is illegal
- The Bitcoin puzzles in this repository are intentionally created challenges with published solutions
- Modern Bitcoin uses 256-bit keys, which are computationally infeasible to break even with this tool

### Understanding Bitcoin Security

- Full 256-bit private keys are secure against Kangaroo attacks
- This tool only works on **restricted ranges** (known subsets)
- Real-world Bitcoin security remains intact
- The puzzles demonstrate the importance of full key space usage

## Section 14: Advanced Topics

### DP Bit Selection Strategy

Choosing the right DP bits is crucial:

```python
# Recommended DP bits based on range size
dp_recommendations = {
    (40, 50): 8,
    (50, 60): 10,
    (60, 70): 12,
    (70, 80): 14,
    (80, 90): 16,
    (90, 100): 18,
    (100, 110): 20,
    (110, 120): 25,
}
```

### Server Split Mode (`-wsplit`)

For very large ranges, use split mode to manage memory:

```bash
# Server saves and resets hash table periodically
./kangaroo -s -d 10 -w save.work -wsplit -wi 600 in.txt

# Later, merge all split files
./kangaroo -wmdir ./workfiles merged.work
```

### Network Security

⚠️ The server has **no authentication mechanism**. Recommendations:

- Run on trusted private networks only
- Use VPN or SSH tunneling for remote access
- Consider firewall rules to restrict access
- Don't expose server port to public internet

## Section 15: References and Resources

### Academic Papers

1. **Using Equivalence Classes to Accelerate Solving the Discrete Logarithm Problem in a Short Interval**
   - https://www.iacr.org/archive/pkc2010/60560372/60560372.pdf

2. **Kangaroo Methods for Solving the Interval Discrete Logarithm Problem**
   - https://arxiv.org/pdf/1501.07019.pdf

3. **Factoring and Discrete Logarithms using Pseudorandom Walks**
   - https://www.math.auckland.ac.nz/~sgal018/crypto-book/ch14.pdf

4. **Kangaroos, Monopoly and Discrete Logarithms**
   - https://web.northeastern.edu/seigen/11Magic/KruskalsCount/PollardKangarooMonopoly.pdf

### Related Resources

- **Bitcoin Puzzle Transaction**: https://www.blockchain.com/btc/tx/08389f34c98c606322740c0be6a7125d9860bb8d5cb182c02f98461e5fa6cd15
- **Discussion Thread**: https://bitcointalk.org/index.php?topic=5244940.0
- **Original Repository**: https://github.com/JeanLucPons/Kangaroo
- **This Fork**: https://github.com/drqsatoshi/kangaroo

### SECP256K1 Curve

- **Equation**: y² = x³ + 7 (mod p)
- **Prime**: p = 2^256 - 2^32 - 977
- **Order**: n = FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
- **Used by**: Bitcoin, Ethereum, and many other cryptocurrencies

## Section 16: Conclusion and Next Steps

### What You've Learned

In this notebook, you've explored:

✅ The Elliptic Curve Discrete Logarithm Problem (ECDLP)
✅ Pollard's Kangaroo algorithm and how it works
✅ Building and running the Kangaroo solver
✅ Understanding distinguished points and performance trade-offs
✅ Working with input files and command-line options
✅ Distributed computing architecture
✅ Performance analysis and optimization

### Next Steps

1. **Experiment with Small Ranges**
   - Start with 40-50 bit ranges on CPU
   - Observe performance and memory usage
   - Try different DP bit settings

2. **Set Up GPU Environment** (if available)
   - Install CUDA toolkit
   - Compile with GPU support
   - Compare GPU vs CPU performance

3. **Try Distributed Mode**
   - Set up a server on one machine
   - Connect clients from other machines
   - Observe collaborative solving

4. **Explore the Code**
   - Study the C++ implementation
   - Understand the cryptographic primitives
   - Learn about GPU optimization techniques

### Final Thoughts

The Kangaroo algorithm beautifully demonstrates:
- The mathematical elegance of elliptic curve cryptography
- The importance of key space in cryptographic security
- How parallel computing can solve complex mathematical problems
- The balance between memory usage and computational efficiency

Remember: This is a powerful educational tool. Use it responsibly and ethically! 🦘

## Appendix A: Quick Reference Card

### Build Commands

```bash
# CPU only
make clean && make all -j4

# GPU (adjust ccap for your GPU)
make clean && make gpu=1 CUDA=/usr/local/cuda ccap=75 all -j4
```

### Common Run Commands

```bash
# Basic CPU run
./kangaroo -t 4 input.txt

# GPU run
./kangaroo -gpu -gpuId 0 input.txt

# With work file saving
./kangaroo -t 4 -ws -w save.work -wi 60 input.txt

# Resume from work file
./kangaroo -t 4 -i save.work

# Run specific puzzle
./kangaroo -t 4 -puzzle 120 puzzle32.txt

# Server mode
./kangaroo -s -d 12 -w server.work -wi 300 input.txt

# Client mode
./kangaroo -t 4 -gpu -c server_ip
```

### Input File Format

```
START_RANGE_HEX
END_RANGE_HEX
PUBLIC_KEY_HEX
```

Example:
```
8000000000
FFFFFFFFFF
02E9F43F810784FF1E91D8BC7C4FF06BFEE935DA71D7350734C3472FE305FEF82A
```

## Appendix B: GPU Compute Capabilities

| GPU Model | Compute Capability | ccap Value |
|-----------|-------------------|------------|
| Tesla K80 | 3.7 | 37 |
| GTX 1080 Ti | 6.1 | 61 |
| Tesla V100 | 7.0 | 70 |
| RTX 2080 Ti | 7.5 | 75 |
| Tesla T4 | 7.5 | 75 |
| Tesla A100 | 8.0 | 80 |
| RTX 3090 | 8.6 | 86 |
| RTX 4090 | 8.9 | 89 |
| Tesla L4 | 8.9 | 89 |

To find your GPU's compute capability:
```bash
nvidia-smi --query-gpu=name,compute_cap --format=csv
```